In [1]:
import cv2
import numpy
from PIL import Image as image_main
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor


# Model, threshold score, class labels, and example image - be sure to replace with your own
image_path = '../../vbpl/test_img/1/vbg_1.jpg'
model_path = '../model/detectron2-publaynet/model/model_final.pth'
model_zoo_config_name = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
prediction_score_threshold = 0.7
class_labels = ['text', 'title', 'list', 'table', 'figure']

# Detectron config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_zoo_config_name))
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
cfg.MODEL.NMS_THRESH_SCORE = 0.5
cfg.MODEL.RPN.NMS_THRESH = 0.7
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.DEVICE = 'cpu'
# Detectron predictor
predictor = DefaultPredictor(cfg)

# Open image, and run predictor
img = cv2.imread(image_path)
outputs = predictor(img)

# Debug outputs
instances = outputs["instances"].to("cpu")
pred_boxes = instances.pred_boxes
scores = instances.scores
pred_classes = instances.pred_classes
boxes = []
colors = [(125,125,0), (255,0,255), (255,255,0), (0,255,255), (255,0,0)]
selected_labels = ['title', 'figure']
for i in range(0, len(pred_boxes)):
    box = pred_boxes[i].tensor.numpy()[0].astype(int)
    score = round(float(scores[i].numpy()), 2)
    label_key = int(pred_classes[i].numpy())
    label = class_labels[label_key]
    x1, y1, x2, y2 = box
    if label in selected_labels:
        boxes.append([list(box), label, score, label_key, img[y1:y2, x1:x2]])
        sorted_boxes = sorted(boxes, key=lambda x: (x[0][1], x[0][0]))
# print(sorted_boxes.__len__())
for i, box in enumerate(sorted_boxes):
    x1, y1, x2, y2 = box[0]
    cv2.rectangle(img, (x1,y1), (x2,y2), colors[box[3]], 2)
    cv2.putText(img, box[1] + '-' + str(box[2]) + '-' + str(i), (x1,y1-5), cv2.FONT_HERSHEY_PLAIN, 2, colors[box[3]], 2)
    cv2.imwrite('./test.jpg', img)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
/home/tuyennx/anaconda3/envs/tesseract_VN/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
